<a href="https://colab.research.google.com/github/KAILASHVenkat/Week2TextToneTune/blob/main/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
pip install transformers pandas datasets

In [9]:
import pandas as pd
from transformers import pipeline
from datasets import load_dataset

emotion_classifier = pipeline('text-classification', model='bhadresh-savani/bert-base-go-emotion')

mrpc_dataset = load_dataset('glue', 'mrpc')
mrpc_df = pd.DataFrame({'input_text': mrpc_dataset['train']['sentence1'], 'target_text': mrpc_dataset['train']['sentence2']})

def get_emotions(text):
    result = emotion_classifier(text)
    dominant_emotion = max(result, key=lambda x: x['score'])
    return dominant_emotion['label']

mrpc_df['input_emotion'] = mrpc_df['input_text'].apply(get_emotions)
mrpc_df['target_emotion'] = mrpc_df['target_text'].apply(get_emotions)

mrpc_df = mrpc_df[mrpc_df['input_emotion'] != mrpc_df['target_emotion']]
mrpc_df = mrpc_df[
    (mrpc_df['input_emotion'] != 'neutral') & (mrpc_df['target_emotion'] != 'neutral')]

decreasing_intensity_pairs = mrpc_df[mrpc_df.apply(
    lambda row: emotion_classifier(row['input_text'])[0]['score'] > emotion_classifier(row['target_text'])[0]['score'],
    axis=1)]

increasing_intensity_pairs = mrpc_df.drop(decreasing_intensity_pairs.index)
increasing_intensity_pairs[['input_text', 'target_text']] = increasing_intensity_pairs[
    ['target_text', 'input_text']].reset_index(drop=True)

decreasing_intensity_pairs['input_text'] = decreasing_intensity_pairs.apply(
    lambda row: f"{row['input_emotion']} to {row['target_emotion']}: {row['input_text']}", axis=1)

final_mrpc_dataset = pd.concat([decreasing_intensity_pairs, increasing_intensity_pairs], ignore_index=True)

final_mrpc_dataset.to_csv('output_mrpc_dataset.csv', index=False)


<ipython-input-9-c11b3bce8547>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  decreasing_intensity_pairs['input_text'] = decreasing_intensity_pairs.apply(


In [16]:
mrpc_df1 = pd.read_csv('output_mrpc_dataset.csv')
mrpc_df1.head()

,input_text,target_text,input_emotion,target_emotion
0,""" I picked prostitutes as my victims because I...","It went on : "" I picked prostitutes as my vict...",anger,annoyance
1,"I never thought I 'd write these words , but h...","I never thought I 'd say this , but I almost m...",disappointment,realization
2,"Mr Hadley , who has a new partner and child , ...","In a statement issued by his solicitors , Mr H...",joy,admiration
3,The seventh person charged in the case - Habis...,"The seventh member of the cell , Habis Abdulla...",amusement,approval
4,Bad publicity has already weakened sales which...,Bad publicity surrounding warranties has alrea...,annoyance,disappointment


In [11]:
# Save the DataFrame to a CSV file in Colab
mrpc_df.to_csv('/content/output_mrpc_dataset.csv', index=False)

In [12]:
from google.colab import files
files.download('/content/output_mrpc_dataset.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>